In [83]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
from time import process_time

pd.options.display.max_rows = 150

### Create sqlite database

In [84]:
conn = sqlite3.connect('temp.db')

#### Load from .csv
Put data into `parking_citations` table if it doesn't already exist

In [85]:
df = pd.read_csv('parking_citations_raw.csv')
# df.to_sql('parking_citations', conn)
pd.read_sql('SELECT * FROM parking_citations', conn)

,index,Unnamed: 0,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,...,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,0,0,1103341116,2015-12-21T00:00:00,1251.0,None,NaN,CA,200304.0,None,...,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
1,1,1,1103700150,2015-12-21T00:00:00,1435.0,None,NaN,CA,201512.0,None,...,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
2,2,2,1104803000,2015-12-21T00:00:00,2055.0,None,NaN,CA,201503.0,None,...,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4
3,3,3,1104820732,2015-12-26T00:00:00,1515.0,None,NaN,CA,NaN,None,...,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2
4,4,4,1105461453,2015-09-15T00:00:00,115.0,None,NaN,CA,200316.0,None,...,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,4995,4273834821,2015-12-30T00:00:00,1046.0,VN204,NaN,AZ,NaN,None,...,PA,BL,14501 GILMORE ST,00363,53.0,88.13B+,METER EXP.,63.0,6425965.9,1890974.4
4996,4996,4996,4273834832,2015-12-30T00:00:00,1053.0,VN360,NaN,CA,201606.0,None,...,PA,TN,14508 SYLVAN ST,00363,53.0,88.13B+,METER EXP.,63.0,6425907.7,1889884.4
4997,4997,4997,4273834843,2015-12-30T00:00:00,1056.0,VN812,NaN,CA,201603.0,None,...,PA,GY,14441 DELANO ST,00363,53.0,88.13B+,METER EXP.,63.0,6426349.4,1889152.6
4998,4998,4998,4273834854,2015-12-30T00:00:00,1101.0,VN686,NaN,CA,201610.0,None,...,PA,RD,14341 DELANO ST,00363,53.0,88.13B+,METER EXP.,63.0,6427009.6,1889150.0


---

### top 25 most common 'makes'

#### sqlite

In [86]:
start = process_time()
cursor = conn.execute("""
SELECT
    `Make`,
    count(*)
FROM
    parking_citations
WHERE
    `Make` IS NOT NULL
GROUP BY
    `Make`
ORDER BY
    count(*) DESC
LIMIT
    25
""")
for row in cursor:
    print(row)
print('completed in {:.5f} seconds'.format(process_time()-start))

('TOYT', 407)
('HOND', 319)
('FORD', 245)
('NISS', 170)
('CHEV', 161)
('BMW', 109)
('VOLK', 90)
('MERZ', 85)
('HYUN', 75)
('DODG', 69)
('LEXS', 56)
('TOYO', 54)
('KIA', 50)
('GMC', 49)
('MAZD', 45)
('JEEP', 44)
('AUDI', 40)
('CHRY', 34)
('INFI', 33)
('OTHR', 29)
('ACUR', 29)
('VOLV', 28)
('SUBA', 28)
('MITS', 25)
('CADI', 22)
completed in 0.00763 seconds


### Pandas

In [87]:
start = process_time()
print(df['Make'].value_counts()[:25])
print('completed in {:.5f} seconds'.format(process_time()-start))

TOYT    816
HOND    609
FORD    464
CHEV    375
NISS    354
BMW     210
VOLK    181
MERZ    166
HYUN    148
DODG    135
LEXS    110
KIA     103
TOYO     95
AUDI     89
GMC      88
MAZD     83
JEEP     82
OTHR     76
CHRY     71
INFI     69
ACUR     63
SUBA     53
VOLV     51
MITS     51
CADI     42
Name: Make, dtype: int64
completed in 0.00338 seconds


---

### most common 'Color' for each 'Make' 

#### sqlite

In [88]:
start = process_time()
cursor = conn.execute("""
SELECT
    `Make`,
    `Color`,
    MAX(count)
FROM
(
    SELECT
        `Make`,
        `Color`,
        count(*) AS count
    FROM
        parking_citations
    WHERE
        `Make` IS NOT NULL
    GROUP BY
        `Make`,
        `Color`
    ORDER BY
        `Make`,
        count(*) DESC
)
GROUP BY
    `Make`
""")
for row in cursor:
    print(row)
print('completed in {:.5f} seconds'.format(process_time()-start))

('ACUR', 'SL', 10)
('AUDI', 'BK', 20)
('BENT', 'GN', 1)
('BENZ', 'BK', 1)
('BMW', 'BK', 35)
('BUIC', 'GY', 3)
('CADI', 'BK', 9)
('CHEV', 'BK', 40)
('CHRY', 'WT', 8)
('DATS', 'OT', 1)
('DODG', 'WT', 17)
('FIAT', 'WT', 1)
('FORD', 'WT', 66)
('FREI', 'RE', 1)
('FRHT', 'WT', 8)
('GEO', 'WT', 2)
('GMC', 'WT', 19)
('GRUM', 'BN', 9)
('HD', 'BK', 1)
('HINO', 'WH', 1)
('HNO', 'WT', 1)
('HOND', 'GY', 78)
('HUMM', 'SL', 1)
('HYUN', 'BK', 19)
('INFI', 'WT', 8)
('INTL', 'RD', 1)
('ISU', 'OR', 1)
('JAGR', 'WH', 1)
('JAGU', 'SL', 1)
('JEEP', 'BK', 16)
('JENS', 'BK', 1)
('KAWK', 'GN', 1)
('KIA', 'BK', 16)
('KW', 'RE', 1)
('LEXS', 'BK', 16)
('LEXU', 'BK', 4)
('LINC', 'BK', 7)
('LNDR', 'BK', 2)
('LROV', 'RE', 1)
('MACK', 'WT', 1)
('MASE', 'WT', 1)
('MAZD', 'GY', 13)
('MBNZ', 'WH', 3)
('MERC', 'GY', 3)
('MERZ', 'BK', 30)
('MITS', 'WT', 11)
('MNNI', 'WT', 7)
('NISS', 'BK', 36)
('OLDS', 'WT', 1)
('OTHR', 'BN', 13)
('PLYM', 'WT', 2)
('PONT', 'WT', 2)
('PORS', 'GY', 4)
('PTRB', 'RD', 1)
('ROL', 'BK', 1)
('RR

### Pandas

In [89]:
start = process_time()
print(df.groupby(['Make'])['Color'].agg(pd.Series.mode))
print('completed in {:.5f} seconds'.format(process_time()-start))

Make
ACC                       GY
ACUR                      BK
AUDI                      BK
BENT                      GN
BENZ                [BK, SI]
BMW                       BK
BUIC                      GY
CADI                      BK
CHEV                      BK
CHRY                      GY
DAEW                      SL
DATS                      OT
DODG                      WT
FIAT                [BL, WT]
FORD                      WT
FREI                      RE
FRHT                      WT
GEO                       WT
GMC                       WT
GRUM                      BN
HD                        BK
HINO                      WH
HNO                       WT
HOND                      GY
HUMM                      BK
HYUN                      BK
INFI                      GY
INTL                      RD
ISU                       WT
ISUZ                      TA
JAGR                      WH
JAGU        [BK, BL, GY, WT]
JEEP                      BK
JENS                      BK
KAWK     

---

### first ticket issued for each 'Make'

#### sqlite

In [90]:
start = process_time()
cursor = conn.execute("""
SELECT
    MIN(`Issue Date`),
    `Make`
FROM
    parking_citations
WHERE
    `Make` IS NOT NULL
GROUP BY
    `Make`
ORDER BY
    `Make`
""")
for row in cursor:
    print(row)
print('completed in {:.5f} seconds'.format(process_time()-start))

('2015-12-21T00:00:00', 'ACUR')
('2015-12-15T00:00:00', 'AUDI')
('2015-12-30T00:00:00', 'BENT')
('2015-12-27T00:00:00', 'BENZ')
('2015-12-08T00:00:00', 'BMW')
('2015-12-28T00:00:00', 'BUIC')
('2015-09-14T00:00:00', 'CADI')
('2015-09-15T00:00:00', 'CHEV')
('2015-12-21T00:00:00', 'CHRY')
('2015-12-30T00:00:00', 'DATS')
('2015-12-12T00:00:00', 'DODG')
('2015-12-30T00:00:00', 'FIAT')
('2015-12-09T00:00:00', 'FORD')
('2015-12-16T00:00:00', 'FREI')
('2015-12-30T00:00:00', 'FRHT')
('2015-12-30T00:00:00', 'GEO')
('2015-12-16T00:00:00', 'GMC')
('2015-12-30T00:00:00', 'GRUM')
('2015-12-30T00:00:00', 'HD')
('2015-12-21T00:00:00', 'HINO')
('2015-12-30T00:00:00', 'HNO')
('2015-09-14T00:00:00', 'HOND')
('2015-12-30T00:00:00', 'HUMM')
('2015-12-22T00:00:00', 'HYUN')
('2015-12-17T00:00:00', 'INFI')
('2015-12-30T00:00:00', 'INTL')
('2015-12-30T00:00:00', 'ISU')
('2015-12-27T00:00:00', 'JAGR')
('2015-12-30T00:00:00', 'JAGU')
('2015-12-09T00:00:00', 'JEEP')
('2015-12-30T00:00:00', 'JENS')
('2015-12-30T00

### Pandas

In [91]:
start = process_time()
print(df.groupby(['Make'])['Issue Date'].agg('min'))
print('completed in {:.5f} seconds'.format(process_time()-start))

Make
ACC     2015-12-23T00:00:00
ACUR    2015-12-21T00:00:00
AUDI    2015-12-15T00:00:00
BENT    2015-12-30T00:00:00
BENZ    2015-12-27T00:00:00
BMW     2015-12-08T00:00:00
BUIC    2015-12-19T00:00:00
CADI    2015-09-14T00:00:00
CHEV    2015-09-15T00:00:00
CHRY    2015-12-19T00:00:00
DAEW    2015-12-30T00:00:00
DATS    2015-12-30T00:00:00
DODG    2015-12-12T00:00:00
FIAT    2015-12-30T00:00:00
FORD    2015-12-09T00:00:00
FREI    2015-12-16T00:00:00
FRHT    2015-12-30T00:00:00
GEO     2015-12-30T00:00:00
GMC     2015-12-16T00:00:00
GRUM    2015-12-30T00:00:00
HD      2015-12-30T00:00:00
HINO    2015-12-21T00:00:00
HNO     2015-12-30T00:00:00
HOND    2015-09-14T00:00:00
HUMM    2015-12-30T00:00:00
HYUN    2015-12-22T00:00:00
INFI    2015-12-17T00:00:00
INTL    2015-12-30T00:00:00
ISU     2015-12-30T00:00:00
ISUZ    2015-12-22T00:00:00
JAGR    2015-12-27T00:00:00
JAGU    2015-12-30T00:00:00
JEEP    2015-12-09T00:00:00
JENS    2015-12-30T00:00:00
KAWK    2015-12-30T00:00:00
KIA     2015-12